<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Decision-Tree" data-toc-modified-id="Decision-Tree-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Decision Tree</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Задача: 
1) Обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

2) Построить модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импорт библиотек и загрузка данных.

In [2]:
import re

import pandas as pd
import nltk
import spacy

from nltk.corpus import stopwords as nltk_stopwords
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import (
    train_test_split,
    cross_val_score)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

In [4]:
try:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col=0)
except:
    df = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)

___

Изучим данные.

In [5]:
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159449,And it looks like it was actually you who put ...,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [7]:
# посмотрим на баланс классов
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Негативных комментариев значительно меньше, чем позитивных. Т. к. мы используем F-1 метрику, которая подходит для оценки предсказания по несбалансированным классам, балансировать классы не обязательно. Однако мы учтем веса классов при обучении моделей и, возможно, подстроим порог классификации логистической регрессии.

___

Далее необходимо токенизировать и лемматизировать тексты, очистить их от ненужных символов, создать мешок слов и рассчитать значения TF-IDF для каждого слова в каждом тексте. Обучать модели будем на этих данных.

Для лемматизации текста воспользуемся библиотекой spacy.

In [8]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# напишем функцию для токенизации и лемматизации
def tokenize_lemmatize(text):
    # лемматизация
    lemmas = nlp(text)
    lemmas = [token.lemma_ for token in lemmas]
    # избавляемся от стоп-слов
    lemmas = [word for word in lemmas if not word in nltk_stopwords.words('english')]
    lemmas = ' '.join(lemmas)
    
    return lemmas

In [9]:
%%time
df['lemm_text'] = df['text'].apply(tokenize_lemmatize)

CPU times: total: 1h 22min 36s
Wall time: 1h 23min 52s


In [10]:
df

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation \n edit make username Hardcore Met...
1,D'aww! He matches this background colour I'm s...,0,D'aww ! match background colour I seemingly st...
2,"Hey man, I'm really not trying to edit war. It...",0,"hey man , I really try edit war . guy constant..."
3,"""\nMore\nI can't make any real suggestions on ...",0,""" \n More \n I make real suggestion improvemen..."
4,"You, sir, are my hero. Any chance you remember...",0,", sir , hero . chance remember page ?"
...,...,...,...
159446,""":::::And for the second time of asking, when ...",0,""" : : : : : second time asking , view complete..."
159447,You should be ashamed of yourself \n\nThat is ...,0,ashamed \n\n horrible thing put talk page . ...
159448,"Spitzer \n\nUmm, theres no actual article for ...",0,"Spitzer \n\n Umm , actual article prostitution..."
159449,And it looks like it was actually you who put ...,0,look like actually put speedy first version de...


Далее очистим текст от нелатинских букв и прочих символов с помощью регулярных выражений.

In [11]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z \']', ' ', text)
    # избавляемся от пробелов:
    text = text.split()
    text = ' '.join(text)
    
    return text

In [12]:
df['lemm_text'] = df['lemm_text'].apply(clear_text)

In [13]:
df

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation edit make username Hardcore Metall...
1,D'aww! He matches this background colour I'm s...,0,D'aww match background colour I seemingly stic...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man I really try edit war guy constantly r...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I make real suggestion improvement I wond...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page
...,...,...,...
159446,""":::::And for the second time of asking, when ...",0,second time asking view completely contradict ...
159447,You should be ashamed of yourself \n\nThat is ...,0,ashamed horrible thing put talk page
159448,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer Umm actual article prostitution ring C...
159449,And it looks like it was actually you who put ...,0,look like actually put speedy first version de...


Разделим выборки на обучающую и тестовую с размером тестовой 20%.

In [14]:
X = df.drop(columns='toxic')
y = df['toxic']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X['lemm_text'], y, test_size=0.2, random_state=0)

## Обучение

### Логистическая регрессия

В качестве финальной предобработки воспользуемся функцией TfidfVectorizer библиотеки sklearn, объединяющую функции создания мешка слов и расчета TF-IDF.

Добавим эту функцию в пайплайн.

In [16]:
stopwords = nltk_stopwords.words('english')

lr_pipe = Pipeline([
    ('TF-IDF', TfidfVectorizer(stop_words=stopwords)),
    ('lr_model', LogisticRegression(random_state=0, class_weight='balanced', max_iter=200))
])

f1_lr_cv = cross_val_score(lr_pipe, X_train, y_train, scoring='f1', cv=5, n_jobs=-1).mean()
print('F-1 логистической регрессии на кросс-валидации:', f1_lr_cv)

F-1 логистической регрессии на кросс-валидации: 0.7496882060328357


### Decision Tree

In [17]:
%%time

dt_pipe = Pipeline([
    ('TF-IDF', TfidfVectorizer(stop_words=stopwords)),
    ('dt_model', DecisionTreeClassifier(random_state=0))
])

f1_dt_cv = cross_val_score(dt_pipe, X_train, y_train, scoring='f1', cv=3, n_jobs=-1).mean()
print('F-1 решающего дерева на кросс-валидации:', f1_dt_cv)

F-1 решающего дерева на кросс-валидации: 0.7029859025088548
CPU times: total: 359 ms
Wall time: 4min 34s


В качестве финальной модели будем использовать логистическую регрессию, т. к. у нее выше значение F-1 метрики на кросс-валидации.

## Выводы

Рассчитаем значение F-1 меры и precision/recall на предсказаниях логистической регрессии на тестовом датасете.

In [18]:
lr_pipe.fit(X_train, y_train)
predictions_test = lr_pipe.predict(X_test)
precision_test = precision_score(y_test, predictions_test)
recall_test = recall_score(y_test, predictions_test)
f1_test = f1_score(y_test, predictions_test)

print('Значение F-1 метрики на тестовой выборке:', round(f1_test, 3))
print('Значение precision на тестовой выборке:', round(precision_test, 2))
print('Значение recall на тестовой выборке:', round(recall_test, 2))

Значение F-1 метрики на тестовой выборке: 0.755
Значение precision на тестовой выборке: 0.67
Значение recall на тестовой выборке: 0.86


Мы выполнили поставленные в проекте задания:
1) Обучили модель логистической регрессии классифицировать комментарии на позитивные и негативные.

2) Построили модель со значением метрики качества *F1* выше 0.75. 